In [1]:
import torch
import torch.nn as nn

def nin_block(in_channels, out_channels, kernel_size, strides, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, strides, padding),
        nn.ReLU(), nn.Conv2d(out_channels, out_channels, kernel_size=1),
        nn.ReLU(), nn.Conv2d(out_channels, out_channels, kernel_size=1),
        nn.ReLU()
    )

In [2]:
net = nn.Sequential(
    nin_block(1, 96, kernel_size=11, strides=4, padding=0),
    nn.MaxPool2d(3, stride=2),
    nin_block(96, 256, kernel_size=5, strides=1, padding=2),
    nn.MaxPool2d(3, stride=2),
    nin_block(256, 384, kernel_size=3, strides=1, padding=1),
    nn.MaxPool2d(3,stride=2), nn.Dropout(0.5),
    nin_block(384, 10, kernel_size=3, strides=1, padding=1),
    nn.AdaptiveAvgPool2d((1,1)),
    nn.Flatten()
)

In [5]:
X= torch.randn(1,1,224,224)

def look_in_net(net, X):
    out = X
    for layer in net:
        out = layer(out)
        print(f"For {layer.__class__.__name__} : ", out.shape)
    

In [6]:
look_in_net(net, X)

For Sequential :  torch.Size([1, 96, 54, 54])
For MaxPool2d :  torch.Size([1, 96, 26, 26])
For Sequential :  torch.Size([1, 256, 26, 26])
For MaxPool2d :  torch.Size([1, 256, 12, 12])
For Sequential :  torch.Size([1, 384, 12, 12])
For MaxPool2d :  torch.Size([1, 384, 5, 5])
For Dropout :  torch.Size([1, 384, 5, 5])
For Sequential :  torch.Size([1, 10, 5, 5])
For AdaptiveAvgPool2d :  torch.Size([1, 10, 1, 1])
For Flatten :  torch.Size([1, 10])


In [8]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.datasets as datasets



my_transforms = transforms.Compose(
                [
                    transforms.Resize(224),
                    transforms.ToTensor()]
)

train_dataset = datasets.FashionMNIST(download=False,root="../data", train=True, transform=my_transforms)
test_dataset = datasets.FashionMNIST(download=False, root="../data", train=False, transform=my_transforms)


train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=True)

In [13]:
import matplotlib.pyplot as plt
    
def train_net(net,num_epochs=5, lr =0.001):
    def accuracy(y_hat,y):
        return (y_hat.argmax(1)==y).sum()
    
    def full_accuracy(net, data_iter):
        net.eval()
        #     device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'
        net = net.to(device)

        total_acc = 0
        total_num = 0
        with torch.no_grad():
            for X, y in data_iter:
                X = X.to(device)
                y = y.to(device)

                y_hat = net(X)

                total_acc += accuracy(y_hat, y)
                total_num += y.numel()

        return total_acc/total_num
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    
    
    
    train_loss = []
    train_acc = []
    test_acc = []

    net= net.to(device)
    
    for epoch in range(num_epochs):
        acc_value = 0
        total_number = 0
        total_loss= 0
        for i, data in enumerate(train_dataloader):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()


            total_loss += loss.item()
            acc_value += accuracy(outputs, labels)
            total_number += labels.numel()

        with torch.no_grad():

            print(f"\tEpoch {epoch} : Statistics: ")
            print(f'\tcurrent train loss : {total_loss} / {total_number} : {float(total_loss/total_number)}')
            print(f'\tcurrent train acc : {acc_value}/{total_number} : {float(acc_value/total_number)}')
            print(f'\tcurrent test acc : {float(full_accuracy(net, test_dataloader))}')


            train_loss.append(float(total_loss/total_number))
            test_acc.append(float(full_accuracy(net, test_dataloader)))
            train_acc.append(float(acc_value/total_number))
    
    print("finished")

    num_epochs = num_epochs
#     plt.plot(range(num_epochs), train_loss, label='train loss')
    plt.plot(range(num_epochs), train_acc, label = 'train acc')
    plt.plot(range(num_epochs), test_acc, label = 'test acc')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    plt.plot(range(num_epochs), train_loss, label='train loss')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    

In [15]:
# try:
#     train_net(net)
# except Exception as e:
#     print(e)
    
# its taking a longtime lets rather try on simler network

	Epoch 0 : Statistics: 
	current train loss : 1081.189864397049 / 60000 : 0.01801983107328415
	current train acc : 6000/60000 : 0.10000000149011612
	current test acc : 0.09999999403953552
	Epoch 1 : Statistics: 
	current train loss : 1081.0921037197113 / 60000 : 0.018018201728661854
	current train acc : 6000/60000 : 0.10000000149011612
	current test acc : 0.09999999403953552


KeyboardInterrupt: 

In [ ]:
my_transforms = transforms.Compose(
                [
                    transforms.ToTensor()
                ]
)

train_dataset = datasets.FashionMNIST(download=False,root="../data", train=True, transform=my_transforms)
test_dataset = datasets.FashionMNIST(download=False, root="../data", train=False, transform=my_transforms)


train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=True)

In [25]:
net = nn.Sequential(nn.Conv2d(1, 96, kernel_size=5, stride=2,padding=1),
                    nn.ReLU(), 
                    nn.Conv2d(96, 48, kernel_size=5, stride=2,padding=1),
                    nn.ReLU(), 
                    
                    nn.Flatten(),
                    nn.Linear(96 * 7 * 7, 40),nn.ReLU(), nn.Dropout(p=0.5),
                    nn.Linear(40, 10)
                    )

In [26]:
look_in_net(net,X)

For Conv2d :  torch.Size([1, 96, 111, 111])
For ReLU :  torch.Size([1, 96, 111, 111])
For Conv2d :  torch.Size([1, 48, 55, 55])
For ReLU :  torch.Size([1, 48, 55, 55])
For Flatten :  torch.Size([1, 145200])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x145200 and 4704x40)